In [3]:
from censai import RIMSharedUnetv3, PhysicalModelv2
from censai.models import SharedUnetModelv4
import tensorflow as tf
import json, os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import matplotlib.pylab as pylab


plt.style.use("science")
params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (10, 10),
         'axes.labelsize': 25,
         'axes.titlesize':25,
         'xtick.labelsize':25,
         'ytick.labelsize':25}
pylab.rcParams.update(params)

# from reproject import reproject_interp

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

np.set_printoptions(edgeitems=10)
np.core.arrayprint._line_width = 180
%matplotlib inline

In [5]:
phys = PhysicalModelv2( # use same parameter as was used during training, rescale kappa later
    pixels=175,
    kappa_pixels=128,
    src_pixels=128,
    image_fov=7.68,
    kappa_fov=7.68,
    src_fov=4,
    method="fft",
)

In [6]:
path = "/media/alexandre/Seagate/Ecole/Automne2021/Censai/models/RIMSU128hstv4_augmented_003_K3_L5_BCL2_211124140837_continue_lr6e-05_211129202839"
with open(os.path.join(path, "unet_hparams.json")) as f:
    unet_params = json.load(f)
# unet_params["kernel_l2_amp"] = args.l2_amp
unet = SharedUnetModelv4(**unet_params)
ckpt = tf.train.Checkpoint(net=unet)
checkpoint_manager = tf.train.CheckpointManager(ckpt, path, 1)
checkpoint_manager.checkpoint.restore(checkpoint_manager.latest_checkpoint).expect_partial()
with open(os.path.join(path, "rim_hparams.json")) as f:
    rim_params = json.load(f)
rim_params["source_link"] = "relu"
rim = RIMSharedUnetv3(phys, unet, **rim_params)

In [ ]:
observation = tf.constant(residual / residual.max(), dtype=tf.float32)[None, ..., None]
lens_light = tf.constant(model, dtype=tf.float32)[None, ..., None]
rim_input1 = tf.constant(rim_input, dtype=tf.float32)[None, ..., None]
psf1 = tf.constant(psf, dtype=tf.float32)[None, ..., None]
noise_rms = tf.constant(std_bkg / residual.max())[None] # same scaling as before  --> make sure rim call support the sigma image
mask1 = tf.constant(mask, dtype=tf.float32)[None, ..., None]